In [ ]:
# hotel_gui.py
from datetime import datetime
import re
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk
from hotel import Hotel, StandardSuite, DeluxeSuite, PresidentialSuite, Customer

hotel = Hotel("The Grand Heritage")

# Add rooms
for i in range(101, 104):
    hotel.add_room(StandardSuite(i))
for i in range(201, 204):
    hotel.add_room(DeluxeSuite(i))
for i in range(301, 304):
    hotel.add_room(PresidentialSuite(i))
    
def book_room():
    name = simpledialog.askstring("Name", "Enter full name:")
    if not name:
        messagebox.showerror("Error", "Name cannot be empty.")
        return

    while True:
        email = simpledialog.askstring("Email", "Enter email:")
        if email and re.match(r"[^@]+@[^@]+\.[^@]+", email):
            break
        messagebox.showerror("Invalid Email", "Please enter a valid email address.")

    while True:
        phone = simpledialog.askstring("Phone", "Enter 10-digit phone:")
        if phone and re.match(r"^[6-9]\d{9}$", phone):
            break
        messagebox.showerror("Invalid Phone", "Please enter a valid 10-digit Indian phone number.")

    id_proof = simpledialog.askstring("ID", "Enter ID proof:")
    if not id_proof:
        messagebox.showerror("Error", "ID proof is required.")
        return

    customer = Customer(name, email, phone, id_proof)
    hotel.add_customer(customer)

    room_type = simpledialog.askstring("Room Type", "1: Standard\n2: Deluxe\n3: Presidential")
    duration = simpledialog.askstring("Duration", "Enter stay duration in days:")
    
    while True:
        checkin = simpledialog.askstring("Check-in Date", "Enter check-in date (dd/mm/yyyy):")
        try:
            datetime.strptime(checkin, "%d/%m/%Y")
            break
        except:
            messagebox.showerror("Invalid Date", "Please use the format dd/mm/yyyy.")

    payment = simpledialog.askstring("Payment Mode", "1: UPI\n2: Debit Card\n3: Credit Card\n4: Net Banking\n5: Cash")

    # Simulate CLI input
    customer_input_func = lambda prompt: {
        "Select Room Type (1/2/3): ": room_type,
        "Enter Check-in Date (dd/mm/yyyy): ": checkin,
        "Enter Stay Duration (in days): ": duration,
        "Enter Payment Mode (1-5): ": payment
    }[prompt]

    import builtins
    original_input = builtins.input
    builtins.input = customer_input_func
    hotel.book_room(customer)
    builtins.input = original_input    

def cancel_booking():
    reg_id = simpledialog.askinteger("Cancel", "Enter Registration ID:")
    room_no = simpledialog.askinteger("Cancel", "Enter Room No:")

    def cancel_input(prompt):
        return str(reg_id if 'registration' in prompt else room_no)

    import builtins
    original_input = builtins.input
    builtins.input = cancel_input
    hotel.cancel_booking()
    builtins.input = original_input

def show_all_bookings():
    if not hotel.bookings:
        messagebox.showinfo("Info", "No bookings available.")
        return

    window = tk.Toplevel()
    window.title("All Bookings")

    cols = ("Reg No", "Customer", "Room No", "Type", "Check-in", "Duration", "Payment")
    tree = ttk.Treeview(window, columns=cols, show="headings")

    for col in cols:
        tree.heading(col, text=col)
        tree.column(col, width=100)

    for b in hotel.bookings:
        tree.insert("", "end", values=(
            b.reg_no,
            b.customer.get_details()["Name"],
            b.room.room_number,
            b.room.__class__.__name__,
            b.check_in,
            b.duration,
            b.payment_mode
        ))

    tree.pack(fill="both", expand=True)

def show_admin_bookings():
    password = simpledialog.askstring("Admin", "Enter admin password:", show='*')
    if password != "3822":
        messagebox.showerror("Error", "Incorrect password")
        return

    if not hotel.bookings:
        messagebox.showinfo("No Bookings", "No bookings available.")
        return

    window = tk.Toplevel()
    window.title("Admin Bookings with Total Amount")

    cols = ("Reg No", "Customer", "Room No", "Type", "Check-in", "Duration", "Payment", "Total ₹")
    tree = ttk.Treeview(window, columns=cols, show="headings")

    for col in cols:
        tree.heading(col, text=col)
        tree.column(col, width=100)

    for b in hotel.bookings:
        total = int(b.duration) * b.room.price
        tree.insert("", "end", values=(
            b.reg_no,
            b.customer.get_details()["Name"],
            b.room.room_number,
            b.room.__class__.__name__,
            b.check_in,
            b.duration,
            b.payment_mode,
            f"Rs.{total}"
        ))

    tree.pack(fill="both", expand=True)

def print_receipt():
    password = simpledialog.askstring("Admin", "Enter admin password:", show='*')
    if password != "3822":
        messagebox.showerror("Error", "Incorrect password")
        return

    if not hotel.bookings:
        messagebox.showinfo("No Bookings", "No bookings available.")
        return

    reg_id = simpledialog.askinteger("Receipt", "Enter Registration ID:")
    try:
        hotel.generate_pdf_receipt(reg_id)
        messagebox.showinfo("Success", f"Receipt for Reg ID {reg_id} generated.")
    except Exception as e:
        messagebox.showerror("Error", str(e))


        
def show_room_hierarchy():
    def display_tree(node, indent=""):
        values.append(indent + str(node.value))
        for child in node.children:
            display_tree(child, indent + "    ")

    values = []
    display_tree(hotel.room_tree)

    hierarchy_window = tk.Toplevel()
    hierarchy_window.title("Room Hierarchy")
    tk.Label(hierarchy_window, text="\n".join(values), justify="left", font=("Courier", 12)).pack(padx=10, pady=10)
    
def show_cancellation_log():
    if not hotel.cancellation_stack:
        messagebox.showinfo("Info", "No cancellations yet.")
        return

    log_window = tk.Toplevel()
    log_window.title("Cancellation Log")

    text = tk.Text(log_window, wrap="word", width=60, height=20)
    text.pack(padx=10, pady=10)

    for b in reversed(hotel.cancellation_stack):
        text.insert(tk.END, f"{b}\n\n")

    text.config(state="disabled")

# GUI setup
root = tk.Tk()
root.title("Hotel Booking System")
root.geometry("400x400")

tk.Label(root, text="The Grand Heritage Hotel", font=("Arial", 16)).pack(pady=10)

tk.Button(root, text="Book Room", width=25, command=book_room).pack(pady=5)
tk.Button(root, text="Cancel Booking", width=25, command=cancel_booking).pack(pady=5)
tk.Button(root, text="Show All Bookings", width=25, command=show_all_bookings).pack(pady=5)
tk.Button(root, text="Show Bookings (Admin)", width=25, command=show_admin_bookings).pack(pady=5)
tk.Button(root, text="Show Room Hierarchy", width=25, command=show_room_hierarchy).pack(pady=5)
tk.Button(root, text="Show Cancellation Log", width=25, command=show_cancellation_log).pack(pady=5)
tk.Button(root, text="Print Receipt (Admin)", width=25, command=print_receipt).pack(pady=5)
tk.Button(root, text="Exit", width=25, command=root.quit).pack(pady=20)

root.mainloop()



Room Types Available:
1. Standard Suite (₹3000)
2. Deluxe Suite (₹5000)
3. Presidential Suite (₹7500)

Choose Payment Mode:
1. UPI
2. Debit Card
3. Credit Card
4. Net Banking
5. Cash

✅ Booking Confirmed!
👉 Registration No: 1010
🏨 Room: 301 | Type: PresidentialSuite | Paid via Cash

❌ No booking found for the given Reg ID and Room Number.

❌ Booking with Reg ID 1010 for Room 301 has been cancelled.

❌ Invalid input. Please enter numeric values.

❌ No booking found for the given Reg ID and Room Number.
